<a href="https://colab.research.google.com/github/iftt-jonathan/database-experiment-colabs/blob/main/mongo/Step7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This will tell you your current IP address from google colab
!curl api.ipify.org

34.148.147.153

In [2]:
# Install needed python packages
%pip install pymongo
%pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 15.6 MB/s eta 0:00:00


In [3]:
# Download us-counties.csv
!curl -L "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv" > us-counties.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 99.9M  100 99.9M    0     0  60.3M      0  0:00:01  0:00:01 --:--:-- 60.2M


In [4]:
import pymongo

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

uri = "mongodb+srv://jonballard6:<password>@cluster0.katr03v.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0"

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)


Pinged your deployment. You successfully connected to MongoDB!


In [5]:
# Create or get your DB
db_name = "CS452_Mongo_Covid"
db = client.get_database(db_name)

Spark SQL Rewrite in MongoDB 1-6

*Redo the SparkSQL assignment in MongoDB using the aggregation pipeline.*

In [19]:
# 1. Write code to define the schema and then read in the dataset
#    (took me 17 minutes!!!)

import pandas

# Load the CSV file
df = pandas.read_csv('./us-counties.csv')
data = df.to_dict('records')
db.casesdeaths.drop()
db.casesdeaths.insert_many(data)
print("done")


done


In [6]:
# 2. Write code to find the county with the most deaths
res = db.casesdeaths.find({},{"_id":0, "state":1, "county":1, "deaths":1}).sort({"deaths":-1}).limit(1)
list(res)

[{'county': 'New York City', 'state': 'New York', 'deaths': 40267.0}]

In [7]:
# 3. Write code to find the county with the most cases
res = db.casesdeaths.find_one({},{"_id":0,"state":1,"county":1,"cases":1},sort=[("cases", -1)])
print(res)

{'county': 'Los Angeles', 'state': 'California', 'cases': 2908425}


In [8]:
# 4. Write code to find the total number of deaths in Utah county
res = db.casesdeaths.find({"county": "Utah"},{"_id":0, "state":1, "county":1, "cases":1}).sort("date", -1).limit(1)
list(res)

[{'county': 'Utah', 'state': 'Utah', 'cases': 203286}]

In [ ]:
# 5. Write code to find the death rate for each state and sort the states by death rate descending
#  with latestDateByFips as (
#       select fips, max(date) as date
#       from covid
#       group by fips
#     ), latestRowForEachFips as (
#       select county, state, cases, deaths, c.fips
#       from covid c
#       join latestDateByFips l on c.date = l.date
#     )

#     select state, (sum(deaths) sum(cases))
#     from latestRowForEachFips
#     group by state

pipeline = [
    {

    },
]
res = db.casesdeaths.aggregate(pipeline)
list(res)

In [ ]:
# 6. Write code to something else interesting with this data – your choice

In this next part we will get experience using MongoDB's aggegregation pipeline's $lookup stage to join collections in MongoDb. Specifically we'll join to our **cases/deaths data** with **[vaccination data](https://ourworldindata.org/us-states-vaccinations#what-share-of-the-population-has-completed-the-initial-vaccination-protocol)** and **[total population data](https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv)**.  First we need to download and ingest the data.



In [4]:
# Get the CSV for covid vaccination data
!curl -L "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/us_state_vaccinations.csv" > "./us_state_vaccinations.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4804k  100 4804k    0     0  10.7M      0 --:--:-- --:--:-- --:--:-- 10.7M


In [ ]:
# Put the vaccinations data into the the DB  (took me 37 seconds)
with open("./us_state_vaccinations.csv") as f:
  dataRows = csv.DictReader(f)
  db.vaccinations.insert_many(dataRows)

df = pandas.read_csv('./us_state_vaccinations.csv')
data = df.to_dict('records')
db.vaccinations.drop()
db.vaccinations.insert_many(data)
print("Done!")


In [5]:
# Get the total population (Use POPESTIMATE2019)
!curl -L "https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv" > "./co-est2019-alldata.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3559k  100 3559k    0     0  8303k      0 --:--:-- --:--:-- --:--:-- 8316k


In [ ]:
# Put population data into the DB (took me 10 seconds)
# with open("./co-est2019-alldata.csv", encoding='latin-1') as f:
#   dataRows = csv.DictReader(f)
#   db.population.insert_many(dataRows)

df = pandas.read_csv('./co-est2019-alldata.csv', encoding='latin-1')
data = df.to_dict('records')
db.populations.drop()
db.populations.insert_many(data)
print("Done!")

Using the aggregation pipeline and the \$out stage create a new dataset that just maps the state to total counts. Do this for all three data sets so you have:

casesdeaths_state = (state, cases, deaths)

populations_state = (state, population)

vaccinations_state = (state, vaccinations)

In [ ]:
# Create the casesdeaths_state collection (remember the counties have a running sum by date, taking the max of each county, then summing by state is correct math)

In [ ]:
# Create the populations_state collection (this dataset is interesting in that there is a "county 0" in each state that represents the state population total)

In [ ]:
# Create the vaccinations_state collection (this dataset is by state and date. You don't want the sum of all the dates, as the data is a running sum)

Use the \$lookup stage of the aggregation pipeline to join your three data sets by state. Note this won't be a perfect join - to find out why look at the states or even the count of states in each set.

In [ ]:
# Report the state, infection rate (cases/population), death rate (deaths/population), vaccination rate (vaccinated_people/population).

In [ ]:
# Is there a correlation between infection or death rates with the vaccination rate for each state?

In [ ]:
# Ask an interesting question that might be answered with this dataset and answer it.